# Mini-ChatGPT

This tutorial assumes the reader has some basic machine learning (ML) and natural language processing (NLP) knowledge, but is new to training large language models, aka foundation models, aka transformer neural networks, using reinforcement learning. If you know what a loss aka cost aka objective function is and how a token aka input_id represents a vector aka embedding, you should be good.

If not here is a really great place to start  https://lena-voita.github.io/nlp_course/language_modeling.html 

Special thanks to Leandro von Werra 's https://github.com/lvwerra/trl repo

diagram from https://openai.com/blog/chatgpt/ ChatGPT is an improved InstructGPT 

<img src='https://cdn.openai.com/chatgpt/draft-20221129c/ChatGPT_Diagram.svg' height=600 width=800>

#### Figure 1

### Step 1

Supervised Fine-Tuning (SFT) is the process of taking a model already pre-trained on a very large corpus of data, and re-training it carefully as not to erase the knowledge the model has gained from that pre-training [catastropic forgetting](https://github.com/clam004/intro_continual_learning), but rather slightly nudge the model to perform better in a more specific or narrow domain represented by a much smaller fine tuning dataset.

There are already plenty of resources on Supervised Fine-Tuning (SFT) aka fine-tuning already, some with impressively good explanations, mistakenly claiming to help you build a mini-chatgpt. I will not cover SFT, instead I will point you to some of my favorite resources. 

Here is an explanatory video by Huggingface of the Decoder, or autoregressive, transformers like GPT: https://youtu.be/d_ixlCubqQw

Here is a great blog explaining language modeling and the cross entropy loss function https://lena-voita.github.io/nlp_course/language_modeling.html , I would start here if you have done ML but not deep learning for NLP before. 

Fine-tuning GPT-like models: https://huggingface.co/course/chapter7/6?fw=pt and [Guide to fine-tuning Text Generation models: GPT-2, GPT-Neo and T5](https://towardsdatascience.com/guide-to-fine-tuning-text-generation-models-gpt-2-gpt-neo-and-t5-dc5de6b3bc5e)

ChatGPT likely was fine-tuned on a dataset of instructions and examples of following to
 
### Step 2

Fine-tuning usually means training for only a small number of epochs with a much smaller learning rate, but also using the cross entropy loss function used to do pre-training, but this way to learning is abit limited, can you tell why? 

<img src="https://lena-voita.github.io/resources/lectures/lang_models/neural/one_step_loss_intuition-min.png" height=500 width=700>

#### Figure 2

This example of using cross entropy shows how the training algorithm rewards or penalizes based on how large the logit (probability mass asigned to) for "cat" is, just this one way of completing the sequence, but we know that in reality when generating responses to instructions, there isnt one right ways to do it even in the cases when there is one right answer, there are many ways to be good and many ways to be bad. 

For the sake of keeping this tutorial light weight, fit on commonly available compute and intuitive, we have simplified the overall strategy in 2 major ways:

A. ChatGPT and InstructGPT, although they are very specially trained with reinforcement learning (RL), when you generalize, ChatGPT is just taking some prompt, aka input text (the phrase or instructions) and generating an output, aka continuing that text (providing a response or answer). It takes alot more compute memory to represent these long instructions and long answers in the neural network transformers, so instead we simplify the input text to the start of a movie review (first few words or subwords) and simplify the continuing text to the continuation of those first few words.

B. The reward modeling has been simplified as compared to InstructGPT https://openai.com/blog/instruction-following/ which is the model that ChatGPT is a scaled up and improved version of. 

Instead of training a reward model based on human rankings of output text, which was done to make the reward signal more stable or reliable, we use the more direct tactic of using the logits from a classifier (like BERT) as the reward signal, "positive meanss do more like this, negative means do less like this", where positive means the review is a referring to movie the reviewer thinks is good (a positive review). 

### Step 3

Given our limited time, I am going to focus on giving you what is hard to find. That is, good explainations of the parts that are usually glossed over yet important, or those parts that are usually explained in a much more jargony, field specific, technical or mathematical manner.

That means we will be focusing on step 3 in Figure 1. Namely, how reinforcement learning, something we have seen thus far in the popular media mostly applied to computer games, instead applied to natural language, or the generation of sequences of discrete tokens. 

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from tqdm.notebook import tqdm
tqdm.pandas()

import torch

from minichatgpt.experiments.imdb import config, sent_kwargs
from minichatgpt.processdata.build_dataset import build_dataset
from minichatgpt import PPOTrainer, LengthSampler, Lab

In [3]:
# For the sake of the speed of this demonstration, the batch_size is temporarily decreased from 256 to 4
batch_size = 4
config.batch_size = batch_size

In [4]:
lab = Lab(config)

dataset = lab.build_dataset(dataset_name="imdb",input_min_text_length=2,input_max_text_length=8)

new_policy, old_policy, tokenizer = lab.init_policies_tokenizer()

lab.set_generation_config(output_min_length=4,output_max_length=16,pad_token_id=tokenizer.eos_token_id)

ppo_trainer = lab.init_ppo_trainer(config, new_policy, old_policy, tokenizer, dataset)

reward_model = lab.init_reward_model()

Found cached dataset imdb (/Users/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1)
Loading cached processed dataset at /Users/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-2bd6a5d7d39a840d.arrow
Loading cached processed dataset at /Users/carson/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1/cache-2ecf25d24c93f132.arrow


#### A new prompt is sampled from the dataset

In the cells below, he text is our samples is represented by the token IDs in input_ids, and each token is either a word or a subword. Roughly speaking the average token represents a word or subword that is around 3 characters long, 'can', 'ed', 'con', 'tion' etc. Because of the code `input_min_text_length=2, input_max_text_length=8` above in `dataset = lab.build_dataset()`, you will find a random assortment of sample lengths in each batch, but they will be no less than 2 tokens short and no more than 8 tokens long. 

In reinforcement learning the agent acts in an environment, like a computer game, and takes actions, like hiting the jump button or moving to the left, which in the future may result in more or less reward, like points in a game. Here the environment is the random assortment of samples that the model can continue is many ways. The action here is how the model chooses to continue those samples, aka prompts.

In [5]:
for batch_step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    
    queries = batch['input_ids']
    
    break
    
print('the part of each batch are: ', batch.keys())
print('-'*50)
print('each batch has ', len(batch['query']), 'samples')
print('-'*50)
print('here are some samples examples ', batch['query'])
print('-'*50)
print('here are the token ids of those examples ', batch['input_ids'])

0it [00:00, ?it/s]

the part of each batch are:  dict_keys(['label', 'input_ids', 'query'])
--------------------------------------------------
each batch has  4 samples
--------------------------------------------------
here are some samples examples  ['I remember watching', "I couldn't hold back", 'A suspenseful thriller that', 'I like musicals']
--------------------------------------------------
here are the token ids of those examples  [tensor([  40, 3505, 4964]), tensor([  40, 3521,  470, 1745,  736]), tensor([   32, 43527,   913, 32251,   326]), tensor([   40,   588, 10530,    82])]


#### The policy generates an output

In [6]:
#### Get response from gpt2
responses = []
for query in queries:
    gen_len = lab.output_length_sampler()
    lab.generation_kwargs["max_new_tokens"] = gen_len
    response = ppo_trainer.generate(query, **lab.generation_kwargs)
    responses.append(response.squeeze()[-gen_len:])
    
responses

[tensor([16089,   351,   337,     5,    38,    11,   290,   484,  6304]),
 tensor([ 379,  477,  287, 4964]),
 tensor([  468,   645,  7110,    11,  3805, 12209, 14586]),
 tensor([ 290, 8842,   11,  290,  314, 1842, 9281,   12,   35, 3383,  518,    7,
         8133, 4480,  257])]

In [8]:
batch['response'] = [tokenizer.decode(r.squeeze()) for r in responses]

concat_text = [batch['query'][i]+" >>> "+batch['response'][i] for i in range(len(batch['response']))]

concat_text

['I remember watching >>>  Raw with M&G, and they weren',
 "I couldn't hold back >>>  at all in watching",
 'A suspenseful thriller that >>>  has no plot, despite occasional grat',
 'I like musicals >>>  and fantasy, and I love Saint-Domingue(!)with a']

I used `>>>` to separate the prompt, left side, which is sampled from the dataset from the text generated by the policy, right side.

#### The reward model calculates a reward for the output

In [9]:
#### Compute sentiment score
texts = [q + r for q,r in zip(batch['query'], batch['response'])]
pipe_outputs = lab.reward_model(texts, **sent_kwargs)
rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]

rewards

/Users/carson/projects/minichatgpt/venv/lib/python3.10/site-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar funcionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[tensor(1.3289), tensor(-1.8032), tensor(0.8134), tensor(2.2469)]

#### The reward is used to update the policy using PPO

PPO means proximal policy optimization. They are the reinforcement learning steps and they are all done by calling `stats = ppo_trainer.step(query_tensors, response_tensors, rewards)` in fact the entire Step 3 training loop is:

```python

for batch_step, batch in tqdm(enumerate(ppo_trainer.dataloader)):
    
    queries = batch['input_ids']
    
    #### Get response from gpt2
    responses = []
    for query in queries:
        gen_len = lab.output_length_sampler()
        lab.generation_kwargs["max_new_tokens"] = gen_len
        response = ppo_trainer.generate(query, **lab.generation_kwargs)
        responses.append(response.squeeze()[-gen_len:])
        
    batch['response'] = [tokenizer.decode(r.squeeze()) for r in responses]
    
    #### Compute sentiment score
    texts = [q + r for q,r in zip(batch['query'], batch['response'])]
    pipe_outputs = lab.reward_model(texts, **sent_kwargs)
    rewards = [torch.tensor(output[1]["score"]) for output in pipe_outputs]
    
    #### Run PPO step 
    stats = ppo_trainer.step(queries, responses, rewards)
    ppo_trainer.log_stats(stats, batch, rewards)

```

But Im here to learn the details, so we will go through PPO as it applies to sequences of discrete tokens, step by step.

In [10]:
# check to make sure that each queries, responses and rewards is a list of tensors
queries, responses, scores = ppo_trainer._step_safety_checker(batch_size, queries, responses, rewards)

#### Policy model architecture 

To be clear on what happens next I find it helpful to remind myself of some of these details about our policy model. First, the policy is a causal language model with an added value head. We will talk about what the value head is later. 

```python

lab.init_policies_tokenizer()

    self.new_policy = AutoModelForCausalLMWithValueHead.from_pretrained(model_name)

```

There are already so many great lessons on the internet about the architectures of transformers, here is a great place to start https://jalammar.github.io/illustrated-gpt2/ so I am not going to explain them here before making reference to them.  

The GPT like models are repeating blocks that pass their hidden state to the next block. The final or last hidden state (aka activations) is used as the input to the language model head (lm_head) and the value head (v_head).

simply run `new_policy` in the next cell to see for yourself

In [ ]:
new_policy

#### Preventing new policy from moving too far from old policy 

Next we find the log probabilities for each token in our responses, notice that the length of each element in the logprobs matches its corresponding pair in responses. We calculate the log probabilities in order to calculate the [Kullback–Leibler divergence](https://machinelearningmastery.com/divergence-between-probability-distributions/) (KL divergence, not because we are implementing the reinforce policy gradient loss function [Policy Gradient Theorem](https://jonathan-hui.medium.com/rl-policy-gradients-explained-9b13b688b146). We are using the [PPO2 algorithm](https://github.com/openai/baselines/tree/master/baselines/ppo2). Using the log probabilities instead to calculate the KL divergence penalty on the reward with expectation $\beta KL(\pi, \rho)$, this penalty is described in equation (2) in [Fine-Tuning Language Models from Human Preferences](https://arxiv.org/pdf/1909.08593.pdf) where $ \pi $ is the distribution of the new policy, $ \rho $ is the distribution of the old policy, aka reference model, aka pretrained transformer, $KL(\pi, \rho)$ is the KL divergence between $ \rho $ and $ \pi $ measured in *nats* and $ \beta $ is dynamically adjusted to acheive a target $KL(\pi, \rho)$ such as *6 nats*.

In [17]:
logprobs, ref_logprobs, values = ppo_trainer.batched_forward_pass(queries, responses)
logprobs

[tensor([-9.1356, -2.0816, -6.7180, -5.0547, -4.3260, -1.9985, -1.2266, -3.6060,
         -5.2811]),
 tensor([-5.0621, -0.4335, -4.2776, -3.4706]),
 tensor([-2.9841, -3.7663, -2.3399, -1.3100, -7.0527, -7.6810, -5.0248]),
 tensor([ -2.0197,  -5.9065,  -1.8272,  -1.7268,  -1.4677,  -2.2598, -11.4134,
          -2.5132,  -3.0124,  -0.3093,  -0.5850,  -6.7908,  -3.5179,  -7.8780,
          -2.0105])]

#### Batched_forward_pass 

Below we dissect the operations done in `batched_forward_pass()` but first a review of some basics.

#### The output logits are policy's actions

If the hidden states size is 768, for example if `new_policy.pretrained_model.lm_head` prints `Linear(in_features=768, out_features=50257, bias=False)`, then that means we are using our last hidden state to predicet logits for each of the 50257 tokens in our vocabulary.

As you can seen below, the outputs of the policy are  `logits, loss, value = model(input_tokens)`

```python
class AutoModelForCausalLMWithValueHead(PreTrainedModelWrapper):
        .
        .
        .
    def forward(
        self,
        input_ids=None,
        past_key_values=None,
        attention_mask=None,
        **kwargs,
    ):
        r"""
        Applies a forward pass to the wrapped model and returns the logits of the value head.
        Args:
            input_ids (`torch.LongTensor` of shape `(batch_size, sequence_length)`):
                Indices of input sequence tokens in the vocabulary.
            past_key_values (`tuple(tuple(torch.FloatTensor))`, `optional`):
                Contains pre-computed hidden-states (key and values in the attention blocks) as computed by the model
                (see `past_key_values` input) to speed up sequential decoding.
            attention_mask (`torch.FloatTensor` of shape `(batch_size, sequence_length)`, `optional`):
                Mask to avoid performing attention on padding token indices. Mask values selected in ``[0, 1]``:
                - 1 for tokens that are **not masked**,
                - 0 for tokens that are **masked**.
            kwargs (`dict`, `optional`):
                Additional keyword arguments, that are passed to the wrapped model.
        """
        base_model_output = self.pretrained_model(
            input_ids=input_ids,
            past_key_values=past_key_values,
            attention_mask=attention_mask,
            output_hidden_states=True,  # We force the model to output hidden states
            **kwargs,
        )

        last_hidden_state = base_model_output.hidden_states[-1]
        lm_logits = base_model_output.logits
        loss = base_model_output.loss

        value = self.v_head(last_hidden_state).squeeze(-1)

        return (lm_logits, loss, value)
```

Logits are basically a non-normalized score for how likely a particular token is to come next. After the logits are pushed through a softmax layer they can sort of be interpreted as probabilities (horizontal bars below).

<img src="https://lena-voita.github.io/resources/lectures/lang_models/neural/nn_lm_idea_linear-min.png" height=600 width=800>

the label "|V| tokens of them" in the diagram above is sayiing that there is a logit for every token in our vocabulary

The shape of the logits tensor is:

(batch_size, token_length_of_longest_sample_in_batch, number_of_tokens_in_vocabulary)

In [42]:
import torch.nn.functional as F

all_logprobs = []
all_ref_logprobs = []
all_values = []

bs = lab.config.batch_size
fbs = lab.config.forward_batch_size

for i in range(int(bs / fbs)):
    
    query_batch = queries[i * fbs : (i + 1) * fbs]
    response_batch = responses[i * fbs : (i + 1) * fbs]
    
    input_ids = ppo_trainer.data_collator([torch.cat([q, r]) for q, r in zip(query_batch, response_batch)])["input_ids"]
    input_kwargs = {"input_ids": input_ids}
    
    with torch.no_grad():
        logits, _, v = new_policy(**input_kwargs)
        ref_logits, _, _ = old_policy(**input_kwargs)

    logp = F.log_softmax(logits[:, :-1, :], dim=2)
    logprobs = torch.gather(logp, 2, input_ids[:, 1:].unsqueeze(2)).squeeze(-1)

    logp = F.log_softmax(ref_logits[:, :-1, :], dim=2)
    ref_logprobs = torch.gather(logp, 2, input_ids[:, 1:].unsqueeze(2)).squeeze(-1)

    for j in range(lab.config.forward_batch_size):

        start = len(query_batch[j]) - 1
        end = len(query_batch[j]) + len(response_batch[j]) - 1

        if len(logprobs[j, start:end]) < 2:
            raise ValueError("Responses are too short. Make sure they are at least 4 tokens long.")

        all_values.append(v[j, start - 1 : end - 1])
        all_logprobs.append(logprobs[j, start:end])
        all_ref_logprobs.append(ref_logprobs[j, start:end])
        
    break
    
all_logprobs

[tensor([-9.1356, -2.0816, -6.7180, -5.0547, -4.3260, -1.9985, -1.2266, -3.6060,
         -5.2811])]

In [55]:
input_ids

tensor([[   40,  3505,  4964, 16089,   351,   337,     5,    38,    11,   290,
           484,  6304]])